In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 100000000)
import warnings



## ETL stage

In [2]:
input_etl = pd.read_csv('input_etl_data.csv', sep = ';')

In [3]:
from src.preprocessing import feature_engineering

In [4]:
fe = feature_engineering.Preprocess()



In [5]:
# Crear una columna con el día de la semana en el que fue creado el ticket
df = fe.datetime_transform(input_etl, 'created_at')

In [6]:
df.head()

,id,external_id,type,subject,priority,status,recipient,requester_id,submitter_id,agent_id,organization_id,group_id,collaborator_ids,collaborators,email_cc_ids,follower_ids,forum_topic_id,problem_id,has_incidents,due_at,tags,via,custom_fields,satisfaction_rating,sharing_agreement_ids,followup_ids,via_followup_source_id,macro_ids,ticket_form_id,brand_id,allow_channelback,allow_attachments,is_public,created_at,updated_at,created_at_day_name,created_at_week
0,1178575,NaN,question,Outras dúvidas,normal,open,atencaoaocliente@deporvillage.com,"3,801E+11","3,801E+11","3,74476E+11",NaN,28479838,[],NaN,[],[],NaN,NaN,False,NaN,"[""deporvillage.com"",""deporvillage.pt"",""other_q...","{""channel"":""email"",""source"":{""from"":{""address""...","[{""id"":114103093293,""value"":""508""},{""id"":11410...","{""score"":""unoffered""}",[],[],NaN,NaN,NaN,3200476,False,True,True,2021-03-15 17:01:18+00:00,2021-03-22T18:22:52Z,Monday,11
1,1186740,NaN,NaN,Re: Tu pedido adsfasdf ha salido del almacén,normal,new,atencionalcliente@deporvillage.zendesk.com,"3,80403E+11","3,80403E+11",NaN,NaN,20938601,[],NaN,[],[],NaN,NaN,False,NaN,"[""deporvillage.com""]","{""channel"":""email"",""source"":{""from"":{""address""...","[{""id"":114103093293,""value"":null},{""id"":114103...","{""score"":""unoffered""}",[],[],NaN,NaN,NaN,3200476,False,True,True,2021-03-22 18:24:19+00:00,2021-03-22T18:24:19Z,Monday,12
2,1180062,NaN,problem,Dfasdfasd - suivi Mondial sdfadsf,normal,solved,fr@deporvillage.zendesk.com,"3,80034E+11","3,80034E+11","3,7489E+11",NaN,21052033,[],NaN,[],[],NaN,NaN,False,NaN,"[""deporvillage.fr"",""otros""]","{""channel"":""email"",""source"":{""from"":{""address""...","[{""id"":114103093293,""value"":""846""},{""id"":11410...","{""score"":""good"",""id"":362032570780,""comment"":""B...",[],[],NaN,NaN,NaN,3200476,False,True,True,2021-03-16 16:03:11+00:00,2021-03-22T18:24:32Z,Tuesday,11
3,1184099,NaN,problem,RE: [Deporvillage] Re: Garantie pour lampe Kno...,normal,open,support@deporvillage.zendesk.com,"3,75277E+11","3,75277E+11","3,74476E+11",NaN,21406268,[],NaN,[],[],NaN,NaN,False,NaN,"[""deporvillage.com"",""otros"",""postventa_fr"",""re...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","[{""id"":114103093293,""value"":""80""},{""id"":114103...","{""score"":""unoffered""}",[],[],NaN,NaN,NaN,3200476,False,True,True,2021-03-19 14:29:24+00:00,2021-03-22T18:24:55Z,Friday,11
4,1177047,NaN,question,Re: [Deporvillage] Re: Re: [Deporvillage] Re: ...,normal,open,it@deporvillage.zendesk.com,"3,74751E+11","3,74751E+11",503374456,NaN,21054107,[],NaN,[],[],NaN,NaN,False,NaN,"[""canvi_talla"",""deporvillage.it"",""tech_questio...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","[{""id"":114103093293,""value"":""174""},{""id"":11410...","{""score"":""unoffered""}",[],[],NaN,NaN,NaN,3200476,False,True,True,2021-03-14 15:42:42+00:00,2021-03-22T18:25:40Z,Sunday,10


## Decodifica la categoría group_id y crea una columna group_des con su descriptivo

'20938601'->'Soporte'

'21052033'->'Francia'

21054107'->'Italia'

'360002394054'->'EN'

'28479838'->'Portugal'

'360002386813'->'DE'

'21406268'->'Postventa'





In [7]:
df.group_id.dtype

df['group_id'] = df.group_id.astype('object')

In [8]:
conditions = [
    (df['group_id'] == 20938601),
    (df['group_id'] == 21052033),
    (df['group_id'] == 21054107),
    (df['group_id'] == 360002394054),
    (df['group_id'] == 28479838),
    (df['group_id'] == 360002386813),
    (df['group_id'] == 21406268)

]

values = ['Soporte', 'Francia', 'Italia', 'EN', 'Portugual', 'DE', 'Postventa']

df['group_des'] = np.select(conditions, values)

In [ ]:
# Extraer 3 columnas nuevas a partir de la columna via

In [9]:
type(df.via)

pandas.core.series.Series

In [24]:
#df['via'] = df.via.to_json(orient=)
import json
df['via'] = df['via'].apply(json.loads)

In [32]:
df['via'] = df['via'].apply(json.dumps)

In [15]:
#df.via.dtype
#df['via_des'] = pd.json_normalize(df.via[0])
pd.io.json.json_normalize(df.via.apply(json.loads))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [10]:
import json 
df.apply(lambda x: json.loads(x['via'])['channel'], axis = 1)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [34]:
df.apply(lambda x: json.loads(x['via'])['channel'], axis = 1)

KeyError: 'channel'

In [35]:
df.apply(lambda x: pd.Series(
    json.loads(x['via']).values(),
    index=json.loads(x['via']).keys()), 
    axis=1)

,0,1,2,3,4,5,6,7,8,9
0,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
1,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
2,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
3,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
4,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
5,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{“channel"":""web"",""source"":{""to"":{},""from"":{""ti...","{""channel"":""web"",""source"":{""from"":{},""to"":{},""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""..."
6,"{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{""channel"":""email"",""source"":{""from"":{""address""...","{“channel"":""web"",""source"":{""to"":{},""from"":{